#### Code to train models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/theo/kaggle/siim_covid/src


## Imports

In [9]:
import os
import pydicom
import numpy as np
import pandas as pd
from PIL import Image

from tqdm.notebook import tqdm

[autoreload of pydicom.data failed: Traceback (most recent call last):
  File "/nix/store/p4xh3dls97j53rscrp6gi4qsc0d6zk4a-python3-3.8.7-env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/nix/store/p4xh3dls97j53rscrp6gi4qsc0d6zk4a-python3-3.8.7-env/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/nix/store/wkw6fsjasr7jbbrlakxxpbiapa8hws42-python3-3.8.7/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/nix/store/wkw6fsjasr7jbbrlakxxpbiapa8hws42-python3-3.8.7/lib/python3.8/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'pydicom' not in sys.modules
]
[autoreload of pydicom._uid_dict failed: Traceback (most recent call last):
  File "/nix/store/p4xh3dls97j53rscrp6gi4qsc0d6zk4a-python3-3.8.7-env/lib/python3.8/site-packages/IPyth

AttributeError: partially initialized module 'pydicom' has no attribute 'uid' (most likely due to a circular import)

In [6]:
from params import *

### Dataset

In [ ]:
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
df = pd.read_csv(DATA_PATH + "train_image_level.csv")

In [ ]:
df.head()

In [ ]:
path = TRAIN_IMG_PATH + 'ae3e63d94c13/288554eb6182/e00f9fe0cce5.dcm'
dicom = pydicom.dcmread(path)

In [ ]:
x = dicom.pixel_array